In [42]:
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [60]:
df = pd.read_excel('D:/UNI/IA/RadioAnalyzer/audios/data.xlsx')

In [61]:
df.head(20)

,audio,categoria
0,BITEL_audio1_1.wav,BITEL
1,BITEL_audio1_2.wav,BITEL
2,BITEL_audio1_3.wav,BITEL
3,BITEL_audio2_1.wav,BITEL
4,BITEL_audio2_2.wav,BITEL
5,BITEL_audio2_3.wav,BITEL
6,BITEL_audio3_1.wav,BITEL
7,BITEL_audio3_2.wav,BITEL
8,BITEL_audio3_3.wav,BITEL
9,CASAMAGIA_audio4_1.wav,CASAMAGIA


In [77]:
N_FCC = 100
def parser(row):
    file_name = 'D:/UNI/IA/RadioAnalyzer/audios/data/'+str(row.audio)
    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=N_FCC).T,axis=0) 
    except Exception as e:
        print("Error encountered while parsing file: ", e)
        return None
 
    feature = mfccs
    return feature

df['feature'] = df.apply(lambda row:parser(row),axis=1)

In [78]:
df.head()

,audio,categoria,feature,predicted,ok
0,BITEL_audio1_1.wav,BITEL,"[-106.61393387377534, 95.37412180539266, -21.3...",BITEL,True
1,BITEL_audio1_2.wav,BITEL,"[-109.58131618811088, 98.67697056184011, -15.6...",BITEL,True
2,BITEL_audio1_3.wav,BITEL,"[-106.4785425144758, 94.28854740407253, -13.79...",BITEL,True
3,BITEL_audio2_1.wav,BITEL,"[-112.08756151856767, 95.88410940638981, -15.6...",BITEL,True
4,BITEL_audio2_2.wav,BITEL,"[-110.18037998963602, 92.20879629345895, -10.5...",BITEL,True


In [79]:
df.feature.mean()

array([-1.34698594e+02,  8.54516728e+01, -6.55805403e+00,  3.43829855e+01,
        6.07344575e+00,  1.81331398e+01, -1.08076713e+01,  1.05620911e+01,
       -1.63225397e+01,  9.27045591e+00, -9.53324571e+00,  6.47441255e+00,
       -8.47810766e+00,  6.29893725e+00, -7.59722708e+00,  5.29705971e+00,
       -1.03307778e+01,  5.62940915e+00, -8.65885338e+00,  3.91779037e+00,
       -8.32280718e+00,  5.07541892e+00, -6.82072170e+00,  5.32696107e+00,
       -5.09310572e+00,  5.13827952e+00, -3.73554836e+00,  4.46792731e+00,
       -3.55870046e+00,  3.28375279e+00, -3.92606099e+00,  3.83466798e+00,
       -2.79150029e+00,  3.08023834e+00, -1.92773175e+00,  1.82241830e+00,
       -1.62724338e+00,  1.30937987e+00, -1.60203557e+00,  8.36826075e-01,
       -2.08910898e+00,  1.02017357e+00, -9.30713692e-01, -1.03813023e-01,
       -8.64384785e-01, -1.58904840e-01, -1.69776343e-01, -4.71453057e-01,
       -5.26642453e-01, -1.53117658e+00,  6.79169623e-01,  1.37690547e-01,
        1.79097550e-01, -

In [107]:
type(df.feature.mean())

numpy.ndarray

In [99]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils

X = np.array(df.feature.tolist())
Y = np.array(df.categoria.tolist())

lb = LabelEncoder()

y = np_utils.to_categorical(lb.fit_transform(Y))


num_labels = y.shape[1]
filter_size = 2

# build model
model = Sequential()

model.add(Dense(256, input_shape=(N_FCC,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [100]:
model.fit(X, y, batch_size=32, epochs=20)

Epoch 1/20
547/547 [==============================] - 1s 1ms/step - loss: 10.7633 - acc: 0.2706
Epoch 2/20
547/547 [==============================] - 0s 64us/step - loss: 10.0502 - acc: 0.3711
Epoch 3/20
547/547 [==============================] - 0s 70us/step - loss: 10.0096 - acc: 0.3547
Epoch 4/20
547/547 [==============================] - 0s 64us/step - loss: 9.7110 - acc: 0.3748
Epoch 5/20
547/547 [==============================] - 0s 66us/step - loss: 9.4427 - acc: 0.3839
Epoch 6/20
547/547 [==============================] - 0s 68us/step - loss: 8.9992 - acc: 0.4022
Epoch 7/20
547/547 [==============================] - 0s 66us/step - loss: 8.7169 - acc: 0.4095
Epoch 8/20
547/547 [==============================] - 0s 68us/step - loss: 8.2821 - acc: 0.4406
Epoch 9/20
547/547 [==============================] - 0s 68us/step - loss: 7.7050 - acc: 0.4442
Epoch 10/20
547/547 [==============================] - 0s 66us/step - loss: 7.5068 - acc: 0.4753
Epoch 11/20
547/547 [================

In [101]:
model.evaluate(X, y)

547/547 [==============================] - 0s 256us/step


[0.39211552385819676, 0.9287020110778879]

In [83]:
predictes_Y = model.predict_classes(X)

In [84]:
predictes_Y_class = lb.inverse_transform(predictes_Y)

In [85]:
df['predicted'] = predictes_Y_class

In [86]:
df.head()

,audio,categoria,feature,predicted,ok
0,BITEL_audio1_1.wav,BITEL,"[-106.61393387377534, 95.37412180539266, -21.3...",BITEL,True
1,BITEL_audio1_2.wav,BITEL,"[-109.58131618811088, 98.67697056184011, -15.6...",BITEL,True
2,BITEL_audio1_3.wav,BITEL,"[-106.4785425144758, 94.28854740407253, -13.79...",BITEL,True
3,BITEL_audio2_1.wav,BITEL,"[-112.08756151856767, 95.88410940638981, -15.6...",BITEL,True
4,BITEL_audio2_2.wav,BITEL,"[-110.18037998963602, 92.20879629345895, -10.5...",BITEL,True


In [87]:
df['ok'] = (df['categoria']==df['predicted'])

In [88]:
df.head()

,audio,categoria,feature,predicted,ok
0,BITEL_audio1_1.wav,BITEL,"[-106.61393387377534, 95.37412180539266, -21.3...",BITEL,True
1,BITEL_audio1_2.wav,BITEL,"[-109.58131618811088, 98.67697056184011, -15.6...",BITEL,True
2,BITEL_audio1_3.wav,BITEL,"[-106.4785425144758, 94.28854740407253, -13.79...",BITEL,True
3,BITEL_audio2_1.wav,BITEL,"[-112.08756151856767, 95.88410940638981, -15.6...",BITEL,True
4,BITEL_audio2_2.wav,BITEL,"[-110.18037998963602, 92.20879629345895, -10.5...",BITEL,True


In [89]:
df['ok'].mean()

0.9396709323583181

In [90]:
dfoutput = df.drop(columns=['feature'])

In [91]:
dfoutput.to_csv("D:/UNI/IA/RadioAnalyzer/audios/out.csv", index=False)

In [109]:
#######SAVING MODEL######
model_json = model.to_json()
with open("D:/UNI/IA/RadioAnalyzer/radioAnalyzerBackend/deeplearning/model.json",'w') as json_file:
    json_file.write(model_json)
model.save_weights("D:/UNI/IA/RadioAnalyzer/radioAnalyzerBackend/deeplearning/model.h5")
##encoder
np.save("D:/UNI/IA/RadioAnalyzer/radioAnalyzerBackend/deeplearning/classes.npy", lb.classes_)

In [105]:
from keras.models import model_from_json
####OPENING MODEL####
json_file = open("D:/UNI/IA/RadioAnalyzer/radioAnalyzerBackend/deeplearning/model.json",'r')
loaded_model_jon = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_jon)

loaded_model.load_weights("D:/UNI/IA/RadioAnalyzer/radioAnalyzerBackend/deeplearning/model.h5")

##evalundo##
loaded_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
score = loaded_model.evaluate(X,y)
print(score[1]*100)

547/547 [==============================] - 0s 506us/step
92.87020110778879
